In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas_profiling import ProfileReport

In [2]:
label_df = pd.read_csv('../data/public_train/train_emotion_labels.csv',header=None,names=['id','image_id','Angry'\
                                                                                                    ,'Disgust','Fear','Happy','Sad',\
                                                                                                    'Surprise','Neutral','Others'])

In [3]:
label_df

,id,image_id,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral,Others
0,NaN,image_id,angry,disgust,fear,happy,sad,surprise,neutral,other
1,0.0,0_19_3,0,0,1,0,0,1,1,0
2,1.0,0_21_4,1,0,1,1,0,0,1,0
3,2.0,0_3_4,0,0,0,1,0,0,1,0
4,3.0,0_35_3,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
6108,6107.0,3829_15_3,1,0,1,0,0,0,0,0
6109,6108.0,3259_15_4,1,1,0,0,0,0,0,0
6110,6109.0,929_19_0,1,0,0,1,0,0,1,0
6111,6110.0,2945_30_1,0,0,1,0,0,1,1,1


In [4]:
label_df.image_id.values

array(['image_id', '0_19_3', '0_21_4', ..., '929_19_0', '2945_30_1',
       '330_37_0'], dtype=object)

In [5]:
label_df.Angry.value_counts()

0        3682
1        2430
angry       1
Name: Angry, dtype: int64

In [6]:
label_df.Disgust.value_counts()

0          3899
1          2213
disgust       1
Name: Disgust, dtype: int64

In [7]:
label_df.Fear.value_counts()

0       3992
1       2120
fear       1
Name: Fear, dtype: int64

In [8]:
label_df.Happy.value_counts()

0        3588
1        2524
happy       1
Name: Happy, dtype: int64

In [9]:
label_df.Sad.value_counts()

0      5180
1       932
sad       1
Name: Sad, dtype: int64

In [10]:
label_df.Surprise.value_counts()

0           4013
1           2099
surprise       1
Name: Surprise, dtype: int64

In [11]:
label_df.Neutral.value_counts()

1          4133
0          1979
neutral       1
Name: Neutral, dtype: int64

In [12]:
label_df.Others.value_counts()

0        5692
1         420
other       1
Name: Others, dtype: int64

In [13]:
len(label_df)

6113

In [14]:
import json

In [15]:
# transcription_df = pd.read_json('../data/public_train/train_transcriptions.json')
with open('../data/public_train/train_transcriptions.json') as file:
    train_transcripts = json.load(file)

In [16]:
transcription_df = pd.DataFrame(train_transcripts)

In [17]:
transcription_df.head()

,dialog,narration,img_id
0,[ the chief mentioned that the place was deser...,[],0_19_3
1,[disappoint you but id probably give you],[],0_21_4
2,[ah i feel better already now for a brisk walk...,[],0_3_4
3,[poor girl ! no wonder shes shy and retiring !],[],0_35_3
4,[gosh guess i tried to civilize these peo a li...,[],1000_16_4


In [18]:
transcription_df.img_id.values

array(['0_19_3', '0_21_4', '0_3_4', ..., '929_19_0', '2945_30_1',
       '330_37_0'], dtype=object)

In [19]:
transcription_df.narration.astype(str).value_counts()

[]                                                                                                                                                                                                                                                                                       4143
['suddenly']                                                                                                                                                                                                                                                                                5
['later']                                                                                                                                                                                                                                                                                   5
['a few minutes later']                                                                                                                       

In [20]:
transcription_df.dialog.astype(str).value_counts()

[]                                                                                                                                                                                           112
[nan]                                                                                                                                                                                         20
['help !']                                                                                                                                                                                     5
['bang !']                                                                                                                                                                                     4
['ugh !']                                                                                                                                                                                      4
                                   

In [21]:
transcription_df['text'] = transcription_df.apply(lambda x: x['dialog']+x['narration'],axis=1)

In [22]:
transcription_df['text'].values

array([list([' the chief mentioned that the place was deserted , but that doesnt seem to be the case !', 'maybe she thinks its a hotel or something !']),
       list(['disappoint you but id probably give you']),
       list(['ah i feel better already now for a brisk walk to calm my jangled nerves !']),
       ...,
       list([' and thats the whole jobin a nuar dap you handle its the first city bank ,... 1f you can try son not to leave as little money r but there as possible you see ithll the 8 now a help my chance of side d becomming the banks pres dent .', 'nut y it souni its darn easy to try fact but since you l the bo - 20 spl now about the side door']),
       list([' only one thing to do , ellen ! draw him into the open - with ourselves as bait ', "i don ' t know maybe you do"]),
       list(['hope this twelve hour hop means action'])], dtype=object)

In [23]:
transcription_df['text'] = transcription_df['text'].apply(lambda x: [y for y in x if type(y) == str])

In [24]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"!", " !", phrase)
    phrase = re.sub(r"\.", " .", phrase)
    phrase = re.sub(r",", " ,", phrase)
    return phrase

In [25]:
from fastpunct import FastPunct

In [26]:
fastpunct = FastPunct()

In [27]:
fastpunct.punct([
                  "i guess the australian government'll be glad to get this enemy base",
                   'disappoint you but id probably give you',
                 '... 1f you can try son not to leave as little money r but there as possible you see ithll the 8 now a help my chance of side d becomming the banks pres dent .',
                 "that dog risked his own life to save him ! gosh that was exciting to watch i almost forgot all about the race ! whos leading ?"], correct=True)

["I guess the Australian government'll be glad to get this enemy base.",
 "Excuse you, but I'd probably give you.",
 "If you can try so not to leave as little money r but there as possible you see it he's now a help my chance of side d becoming the bank president.",
 "That dog risked his own life to save him! Guess that was exciting to watch I almost forgot all about the race! Who's leading?"]

In [28]:
transcription_df['text'][:10].apply(lambda x: fastpunct.punct(x,correct=True)).values

array([list(["The chief mentioned that the place was deserted, but that doesn't seem to be the case!", "Maybe she thinks it's a hotel or something!"]),
       list(["Excuse you, but I'd probably give you."]),
       list(['Ah I feel better already now for a brisk walk to calm my jangled nerves!']),
       list(["Poor girl! no wonder she's shy and retiring!"]),
       list(['Gosh guess I tried to civilize these people a little too fast and now I have to act unoe civilized in self defense!', 'Get the runt!', 'Let me at IIM.']),
       list(["I won't waste another minute.", 'Women Your City is in the grip of a crime wave! Volunteer for police duty!']),
       list(['A magnesium flare! Ohhhh.', "A string is plucked, the minstrel's discharges a small capsule that goes into blinding brilliance,."]),
       list(['Step in line and sink your calks up, men who fall out will beleft behind now.Excelsior!']),
       list(['You said it would be suicide but he made it!', "That dog risked his own lif

In [29]:
transcription_df = transcription_df[transcription_df['text'].astype(str)!='[]']

In [30]:
# transcription_df['text_clean'] = transcription_df['text'].apply(lambda x: fastpunct.punct(x,correct=True))

KeyboardInterrupt: 

In [ ]:
transcription_df[['text','text_clean']]

In [ ]:
transcription_df['text_clean'] = transcription_df['text_clean'].apply(lambda x: [decontracted(y).lower() for y in x])

In [ ]:
transcription_df

In [ ]:
transcription_df.columns = ['dialog', 'narration', 'image_id', 'text','text_clean']

In [ ]:
transcription_df.head()

In [ ]:
len(transcription_df)

In [ ]:
df = pd.merge(label_df,transcription_df,on='image_id',how='inner')

In [ ]:
df.head()

In [ ]:
emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral','Others']

In [ ]:
df['emotion_list'] = df.apply(lambda x: [emo for emo in emotions if x[emo] == '1'],axis=1)

In [ ]:
df.to_csv('../data/public_train/dataset.csv',index=None)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.1)
print(type(train), type(test), type(val))

In [ ]:
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)

In [ ]:
val.to_csv("../data/public_train/val_data.csv",index=None)
test.to_csv("../data/public_train/test_data.csv",index=None)
train.to_csv("../data/public_train/train_data.csv",index=None)

In [2]:
val=pd.read_csv("../data/public_train/val_data.csv")
test=pd.read_csv("../data/public_train/test_data.csv")
train=pd.read_csv("../data/public_train/train_data.csv")

In [3]:
import cv2

def resize_img(path):
  try:
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)/255
    return img
  except Exception as e:
    print(str(e))
    return None

In [4]:
base_path = '../data/public_train/train/'

In [5]:
val_imgs = []
i = 0
for index, row in val.iterrows():
  if i%100 == 0:
    print("Processing i:", i, base_path+str(row['image_id'])+'.jpg')
  img = resize_img(base_path+str(row['image_id'])+'.jpg')
  if img is not None:
    val_imgs.append(img)
  else:
#     val = val.drop(index)
    print("Dropping row:", index, "Length:", len(val))
  i += 1

Processing i: 0 ../data/public_train/train/1179_7_2.jpg
Processing i: 100 ../data/public_train/train/1774_32_5.jpg
Processing i: 200 ../data/public_train/train/3083_44_2.jpg
Processing i: 300 ../data/public_train/train/1584_25_6.jpg
Processing i: 400 ../data/public_train/train/2252_54_2.jpg


In [6]:
assert len(val) == len(val_imgs)

In [7]:
val_np_imgs = np.array(val_imgs)
val_np_imgs.shape

(486, 224, 224, 3)

In [8]:
with open('../data/public_train/val_np_img_norm','wb') as f: pickle.dump(val_np_imgs, f)

In [9]:
train_imgs = []
i = 0
for index, row in train.iterrows():
  if i%100 == 0:
    print("Processing i:", i, base_path+str(row['image_id'])+'.jpg')
  img = resize_img(base_path+str(row['image_id'])+'.jpg')
  if img is not None:
    train_imgs.append(img)
  else:
#     train = train.drop(index)
    print("Dropping row:", index, "Length:", len(train))
  i += 1

Processing i: 0 ../data/public_train/train/1308_48_2.jpg
Processing i: 100 ../data/public_train/train/3592_41_5.jpg
Processing i: 200 ../data/public_train/train/1555_21_3.jpg
Processing i: 300 ../data/public_train/train/1146_21_3.jpg
Processing i: 400 ../data/public_train/train/211_94_3.jpg
Processing i: 500 ../data/public_train/train/714_33_9.jpg
Processing i: 600 ../data/public_train/train/1890_48_1.jpg
Processing i: 700 ../data/public_train/train/1079_22_2.jpg
Processing i: 800 ../data/public_train/train/2257_64_1.jpg
Processing i: 900 ../data/public_train/train/1091_7_0.jpg
Processing i: 1000 ../data/public_train/train/3117_48_6.jpg
Processing i: 1100 ../data/public_train/train/3753_58_3.jpg
Processing i: 1200 ../data/public_train/train/2143_47_7.jpg
Processing i: 1300 ../data/public_train/train/3754_12_3.jpg
Processing i: 1400 ../data/public_train/train/1630_47_5.jpg
Processing i: 1500 ../data/public_train/train/3663_5_2.jpg
Processing i: 1600 ../data/public_train/train/2544_6_0.j

In [10]:
train_np_imgs = np.array(train_imgs)
train_np_imgs.shape

(4365, 224, 224, 3)

In [11]:
with open('../data/public_train/train_np_img_norm','wb') as f: pickle.dump(train_np_imgs, f)

In [12]:
assert len(train) == len(train_imgs)

In [13]:
test_imgs = []
i = 0
for index, row in test.iterrows():
  if i%100 == 0:
    print("Processing i:", i, base_path+str(row['image_id'])+'.jpg')
  img = resize_img(base_path+str(row['image_id'])+'.jpg')
  if img is not None:
    test_imgs.append(img)
  else:
#     test = test.drop(index)
    print("Dropping row:", index, "Length:", len(test))
  i += 1

Processing i: 0 ../data/public_train/train/3812_3_3.jpg
Processing i: 100 ../data/public_train/train/1906_25_6.jpg
Processing i: 200 ../data/public_train/train/2399_62_5.jpg
Processing i: 300 ../data/public_train/train/1514_3_5.jpg
Processing i: 400 ../data/public_train/train/1634_41_5.jpg
Processing i: 500 ../data/public_train/train/567_11_0.jpg
Processing i: 600 ../data/public_train/train/2279_33_3.jpg
Processing i: 700 ../data/public_train/train/966_19_5.jpg
Processing i: 800 ../data/public_train/train/2319_31_1.jpg
Processing i: 900 ../data/public_train/train/3075_27_4.jpg
Processing i: 1000 ../data/public_train/train/2805_31_6.jpg
Processing i: 1100 ../data/public_train/train/1190_23_3.jpg
Processing i: 1200 ../data/public_train/train/1125_7_5.jpg


In [14]:
assert len(test) == len(test_imgs)

In [15]:
test_np_imgs = np.array(test_imgs)
test_np_imgs.shape

(1213, 224, 224, 3)

In [16]:
with open('../data/public_train/test_np_img_norm','wb') as f: pickle.dump(test_np_imgs, f)